Import the necessary python libraries and Susie objects.

In [1]:
%pip install -i https://test.pypi.org/simple/ susie==1.0.3

Looking in indexes: https://test.pypi.org/simple/
You should consider upgrading via the '/Users/maliabarker/Desktop/main/susie/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
from susie.transit_times import TransitTimes
from susie.ephemeris import Ephemeris
import matplotlib.pyplot as plt
from astropy import coordinates as coords
from astropy import units as u
from astropy import time

In [6]:
coors = ('06h30m32.79s +29d40m20.16s')
c = coords.SkyCoord(coors, unit=(u.hourangle, u.deg), obstime="J1992.21")
c.ra

<Longitude 97.636625 deg>

Add your transit data to the TransitTimes object.

In [ ]:
# If you have CSV File, use this
# STEP 1: Upload data from file
filepath = "./malia_examples/WASP12b_transit_ephemeris.csv"
data = np.genfromtxt(filepath, delimiter=',', names=True)
# STEP 2: Break data up into epochs, mid transit times, and error
epochs = data["epoch"] - np.min(data["epoch"])
mid_transit_times = data["transit_time"] - np.min(data["transit_time"])
mid_transit_times_err = data["sigma_transit_time"]

In [ ]:
# If not use this
# Hard Coded Data
epoch_data = [-1640.0, -1346.0, -1342.0, -1067.0, -1061.0, -1046.0, -1038.0, -1004.0, -1003.0, -985.0, -963.0, -743.0, -739.0, -729.0, -728.0, -721.0, -699.0, -699.0, -677.0, -655.0, -648.0, -646.0, -645.0, -643.0, -625.0, -393.0, -383.0, -382.0, -380.0, -368.0, -353.0, -350.0, -329.0, -328.0, -327.0, -324.0, -323.0, -317.0, -316.0, -307.0, -306.0, -296.0, -295.0, -294.0, -293.0, -283.0, -275.0, -274.0, -55.0, -51.0, -29.0, -21.0, -19.0, -7.0, -3.0, 0.0, 13.0, 21.0, 22.0, 274.0, 275.0, 276.0, 277.0, 297.0, 298.0, 320.0, 324.0, 327.0, 328.0, 329.0, 338.0, 341.0, 351.0, 356.0, 365.0, 372.0, 379.0, 381.0, 382.0, 624.0, 646.0, 648.0, 678.0, 679.0, 691.0, 692.0, 698.0, 699.0, 731.0, 953.0, 994.0, 995.0, 1027.0, 1028.0, 1050.0, 1252.0, 1270.0, 1281.0, 1284.0, 1302.0, 1303.0, 1338.0, 1339.0, 1344.0, 1345.0, 1348.0, 1352.0, 1352.0, 1357.0, 1359.0, 1370.0, 1377.0, 1378.0, 1379.0, 1577.0, 1599.0, 1608.0, 1620.0, 1621.0, 1624.0, 1666.0, 1667.0, 1674.0, 1676.0, 1678.0, 1695.0, 1695.0, 1696.0, 1699.0, 1701.0, 1701.0, 1702.0, 1702.0, 1705.0, 1716.0, 1930.0, 1985.0, 2006.0, 2017.0]
mid_transit_time_data = [2454515.525, 2454836.403, 2454840.769, 2455140.91, 2455147.459, 2455163.831, 2455172.561, 2455209.669, 2455210.762, 2455230.407, 2455254.419, 2455494.53, 2455498.896, 2455509.81, 2455510.902, 2455518.541, 2455542.552, 2455542.553, 2455566.564, 2455590.576, 2455598.216, 2455600.398, 2455601.49, 2455603.673, 2455623.318, 2455876.528, 2455887.442, 2455888.533, 2455890.716, 2455903.814, 2455920.184, 2455923.459, 2455946.378, 2455947.47, 2455948.561, 2455951.835, 2455952.927, 2455959.475, 2455960.567, 2455970.389, 2455971.481, 2455982.395, 2455983.487, 2455984.578, 2455985.67, 2455996.584, 2456005.315, 2456006.406, 2456245.427, 2456249.794, 2456273.805, 2456282.536, 2456284.719, 2456297.816, 2456302.182, 2456305.455, 2456319.644, 2456328.376, 2456329.467, 2456604.505, 2456605.596, 2456606.688, 2456607.779, 2456629.607, 2456630.699, 2456654.71, 2456659.076, 2456662.35, 2456663.441, 2456664.533, 2456674.356, 2456677.63, 2456688.544, 2456694.002, 2456703.824, 2456711.464, 2456719.104, 2456721.287, 2456722.378, 2456986.502, 2457010.513, 2457012.696, 2457045.438, 2457046.53, 2457059.627, 2457060.718, 2457067.267, 2457068.358, 2457103.284, 2457345.579, 2457390.327, 2457391.418, 2457426.343, 2457427.435, 2457451.446, 2457671.913, 2457691.559, 2457703.564, 2457706.838, 2457726.484, 2457727.575, 2457765.775, 2457766.866, 2457772.324, 2457773.415, 2457776.689, 2457781.054, 2457781.056, 2457786.512, 2457788.695, 2457800.7, 2457808.34, 2457809.432, 2457810.523, 2458026.624, 2458050.635, 2458060.459, 2458073.555, 2458074.647, 2458077.921, 2458123.76, 2458124.852, 2458132.491, 2458134.675, 2458136.858, 2458155.41, 2458155.412, 2458156.503, 2458159.778, 2458161.96, 2458161.96, 2458163.051, 2458163.051, 2458166.326, 2458178.331, 2458411.895, 2458471.923, 2458494.843, 2458506.848]
mid_transit_times_uncertainties_data = [0.00043, 0.00028, 0.00062, 0.00042, 0.00043, 0.00032, 0.00036, 0.00046, 0.00041, 0.00019, 0.00043, 0.00072, 0.00079, 0.00037, 0.00031, 0.0004, 0.0004, 0.00028, 0.00028, 0.00068, 0.00035, 0.00029, 0.00024, 0.00029, 0.00039, 0.00027, 0.00021, 0.00027, 0.00024, 0.00032, 0.00031, 0.00022, 0.00018, 0.00017, 0.00033, 0.00011, 0.0001, 0.00017, 0.00032, 0.00039, 0.00035, 0.00034, 0.00035, 0.00032, 0.00042, 0.00037, 0.00037, 0.00031, 0.00033, 0.00039, 0.0003, 0.0003, 0.0003, 0.0003, 0.00046, 0.00024, 0.00038, 0.00027, 0.00029, 0.00021, 0.0003, 0.00033, 0.00071, 0.00019, 0.00043, 0.00034, 0.00034, 0.00019, 0.00019, 0.00031, 0.00028, 0.00032, 0.0004, 0.00029, 0.00029, 0.00025, 0.00034, 0.00034, 0.00046, 0.00043, 0.00039, 0.00049, 0.00046, 0.00049, 0.00035, 0.00036, 0.00022, 0.0002, 0.00031, 0.00042, 0.00033, 0.00033, 0.00055, 0.00023, 0.00021, 0.00035, 0.00025, 0.00034, 0.00037, 0.00028, 0.00023, 0.00028, 0.00039, 0.00024, 0.00022, 0.00029, 0.00043, 0.00036, 0.00026, 0.00048, 0.00032, 0.0004, 0.00018, 0.00021, 0.00056, 0.00023, 0.0003, 0.00022, 0.00034, 0.00028, 0.00027, 0.00035, 0.00031, 0.00032, 0.00033, 0.0005, 0.00031, 0.00032, 0.00091, 0.00035, 0.00026, 0.00021, 0.00034, 0.00034, 0.00038, 0.0004, 0.00026, 0.0003, 0.00044]

epochs = epoch_data - np.min(epoch_data)
mid_transit_times = mid_transit_time_data - np.min(mid_transit_time_data)
mid_transit_times_err = mid_transit_times_uncertainties_data

In [10]:
# NOTE: You can use any method and/or file type to upload your data. Just make sure the resulting variables (epoch, mid transit times, and mid transit time errors) are numpy arrays
# STEP 2.5 (Optional): Make sure the epochs are integers and not floats
epochs = epochs.astype('int')
# STEP 3: Create new transit times object with above data
transit_times_obj1 = TransitTimes(epochs, mid_transit_times, mid_transit_times_err)
# STEP 4 (Optional): To check that the resulting data is correct, you can print each variable and the corresponding data using this command: print(vars(transit_times_obj1))

In [11]:
mid_tt = time.Time(mid_transit_times, format='jd', scale=None)

In [18]:
test_arr = np.ones_like(mid_tt, dtype=float)

In [19]:
test_tt = time.Time(test_arr, format='jd', scale=None)

In [20]:
np.all(test_tt.value == 1)

True

In [22]:
earth_center = coords.EarthLocation.from_geocentric(0., 0., 0., unit=u.m)

In [25]:
earth_center.value

(0., 0., 0.)

In [32]:
obs_location = coords.EarthLocation.from_geodetic(-116, 42)

In [38]:
obj_coord = coords.SkyCoord(ra=97.64, dec=29.67, unit='deg', frame='icrs')

In [48]:
print(obj_coord.ra.value)

97.64


Create the Ephemeris object and add your TransitTimes object.

In [ ]:
ephemeris_obj1 = Ephemeris(transit_times_obj1)

Fit your transit time data to an ephemeris model. You can specify what type of model with the options 'linear' or 'quadratic'.

In [ ]:
# Getting a linear model will solve for period and conjuction time (and their respective errors)
linear_model_data = ephemeris_obj1.get_model_ephemeris('linear')

In [ ]:
# Show model fit
plt.scatter(x=transit_times_obj1.epochs, y=linear_model_data, marker='v')
plt.scatter(x=transit_times_obj1.epochs, y=transit_times_obj1.mid_transit_times, marker='o')
plt.show()

In [ ]:
# Getting a quadratic model will solve for period, conjunction time, and period change by epoch (and their respective errors)
quad_model_data = ephemeris_obj1.get_model_ephemeris('quadratic')

In [ ]:
# Show model fit
plt.scatter(x=transit_times_obj1.epochs, y=quad_model_data, marker='v')
plt.scatter(x=transit_times_obj1.epochs, y=transit_times_obj1.mid_transit_times, marker='o')
plt.show()